##### Student Name: Dara Leonard
##### Student ID: 19202478

#### Task 1: Identify one or more suitable web APIs

In [ ]:
import pandas as pd
import requests
import json
from stravalib import Client

strava_client = Client(access_token="338408a3d0e8b210fcd06bd3fea088a2d17aae14 ")
weather_api_key = "4a88876129264886896114557202902"

def build_weather_request(location, year, month, day):
    return "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=" + weather_api_key + "&q="+ location +"&format=json&date="+ str(year) + "-" + str(month) + "-" + str(day)

#### Task 2: Collect data your chosen API(s)

In [ ]:
json_weather_data = []
sw_lat = 53.263329
sw_lng = -9.070642
ne_lat = 53.283310
ne_lng = -9.044810

segments_in_an_area = strava_client.explore_segments(bounds=[sw_lat, sw_lng, ne_lat, ne_lng], activity_type='running') # Get current athlete details
segment_info = [strava_client.get_segment(segment.id) for segment in segments_in_an_area]
 
def execute_request(link):
    response = requests.get(link)
    json_res = response.json()['data']['weather']
    json_weather_data.append(json_res)
 
segment_info_columns = ["id", "name", "activity_type", "distance", "start_latitude", "start_longitude", "end_latitude",
           "end_longitude", "city", "state", "country", "hazardous", "map_id", "map_polyline", "effort_count",
           "athlete_count", "star_count", "best_time", "best_time_date"]
segment_info_df = pd.DataFrame(columns=segment_info_columns)
for i, segment in enumerate(segment_info):
    segment_leader_board = strava_client.get_segment_leaderboard(segment.id)
    best_time = segment_leader_board.entries[0].elapsed_time
    best_time_date = segment_leader_board.entries[0].start_date
    tmp = dict({"id": segment.id,
             "name": segment.name,
             "activity_type": segment.activity_type,
             "distance": segment.distance,
             "start_latitude": segment.start_latitude,
             "start_longitude": segment.start_longitude,
             "end_latitude": segment.end_latitude,
             "end_longitude": segment.end_longitude,
             "city": segment.city,
             "state": segment.state,
             "country": segment.country,
             "hazardous": segment.hazardous,
             "map_id": segment.map.id,
             "map_polyline": segment.map.polyline,
             "effort_count": segment.effort_count,
             "athlete_count": segment.athlete_count,
             "star_count": segment.star_count,
             "best_time" : segment_leader_board.entries[0].elapsed_time, 
             "best_time_date" : segment_leader_board.entries[0].start_date})
    segment_info_df.loc[i] = tmp
        
    try:
        execute_request(build_weather_request(str(segment.start_latitude + segment.start_longitude), best_time_date.year, best_time_date.month, best_time_date.day))
    except KeyError:
        execute_request(build_weather_request(str(sw_lat + sw_lng), best_time_date.year, best_time_date.month, best_time_date.day))

#### Task 3: Parse the collected data, and store it in an appropriate file format

In [ ]:
def write_data_to_file(filename, data_to_write):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data_to_write, file, ensure_ascii=False, indent=4)

write_data_to_file('strava-data.json', segment_info_df.to_json())
write_data_to_file('weather-data.json', json_weather_data)

In [33]:
print("Done")


Done
